<a href="https://colab.research.google.com/github/lizardnote/ML_practice/blob/main/%EB%B9%85%EB%B6%84%EA%B8%B0ch5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

모평균 구간 추정

In [3]:
from scipy.stats import t
import numpy as np

data = [4.3, 4.1, 5.2, 4.9, 5.0, 4.5, 4.7, 4.8, 5.2, 4.6]
mean = np.mean(data)
n = len(data)
se = np.std(data, ddof = 1) / np.sqrt(n)
print(round(mean - t.ppf(0.975, n-1)* se, 3), round(mean + t.ppf(0.975, n-1) * se, 3))

4.469 4.991


In [4]:
ci = t.interval(0.95, loc = mean, scale = se, df = n-1)
print('신뢰구간 : ', [round(i,3) for i in ci])

신뢰구간 :  [np.float64(4.469), np.float64(4.991)]


단일 표본 t검정 - ttest-1samp()

In [2]:
sample = [9.76,11.1,10.7,10.72,11.8,6.15,10.52,14.83,13.03,16.46,10.84,12.45]

from scipy.stats import ttest_1samp

t_statistic, p_value = ttest_1samp(sample, popmean = 10, alternative = 'two-sided')
print(t_statistic, p_value)

2.0508338167773075 0.06488240727465687


독립 2 표본 t 검정 : ttest-iind()

In [3]:
import pandas as pd

sample = [9.76,11.1,10.7,10.72,11.8,6.15,10.52,14.83,13.03,16.46,10.84,12.45]
gender = ['Female']*7 + ["Male"]*5

my_tab2 = pd.DataFrame({'score' : sample, 'gender' : gender})
my_tab2

,score,gender
0,9.76,Female
1,11.10,Female
2,10.70,Female
3,10.72,Female
4,11.80,Female
5,6.15,Female
6,10.52,Female
7,14.83,Male
8,13.03,Male
9,16.46,Male


In [5]:
from scipy.stats import ttest_ind

male = my_tab2[my_tab2['gender'] == 'Male']['score']
female = my_tab2[my_tab2['gender'] == 'Female']['score']


,score
7,14.83
8,13.03
9,16.46
10,10.84
11,12.45


In [10]:
t_statistic, p_value = ttest_ind(male['score'], female['score'], equla_var= True, alternative= 'greater')
print(t_statistic, p_value)

,score
7,14.83
8,13.03
9,16.46
10,10.84
11,12.45


대응 표본 t검정 : ttest_rel()



In [17]:
import numpy as np

before = np.array([9.76,11.1,10.7,10.72,11.8,6.15])
after = np.array(
  [10.52, 14.83, 13.03, 16.46,10.84,12.45])

In [19]:
from scipy.stats import ttest_rel

t_statistic, p_value = ttest_rel(before, after, alternative= 'greater')
print(t_statistic, p_value)

-2.5811614301011883 0.9753187165445341


대응표본t검정을 1표본 t검정으로 바꿔서 해결해본다면

In [20]:
from scipy.stats import ttest_1samp

sample_d = after - before
t_statistic, p_value = ttest_1samp(sample_d, popmean = 0, alternative= 'greater')
print(t_statistic, p_value)


2.5811614301011883 0.02468128345546597


F검정

In [33]:
import numpy as np
import scipy.stats as stats

oj_length = np.array([17.6,9.7,16.5,12,21.5,23.3,23.6,26.4,20,25.2,25.8,21.2,14.5,27.3,23.8])
vc_length = np.array([7.6,4.2,10.0,11.5,7.3,5.8,14.5,10.6,8.2,9.4,16.5,9.7,8.3,13.6,8.2])


s1 = oj_length.std(ddof = 1)
s2 = vc_length.std(ddof = 1)

ratio_df_var = s1**2 / s2**2

round(ratio_df_var,4)

np.float64(2.7224)

In [36]:
def F_test(x, y, alternative='two_sided'):
  x = np.array(x)
  y = np.array(y)
  df1 = len(x) - 1
  df2 = len(y) - 1
  f = np.var(x, ddof = 1) / np.var(y, ddof = 1)
  if alternative == 'greater':
    p = 1.0 - stats.f.cdf(f, df1, df2)
  elif alternative == 'less':
    p = stats.f.cdf(f, df1, df2)
  else:
    p = 2.0 * (1.0 - stats.f.cdf(f, df1, df2))
  return f, p

f_value, p_value = F_test(oj_length, vc_length)
print(f_value, p_value)

2.7223514017700063 0.07111637215506228


사분위수

In [37]:
import numpy as np
data  = np.array([155,126,27,82,115,140,73,92,110,134])

sorted_data = np.sort(data)

min = np.min(sorted_data)
max = np.max(sorted_data)

median = np.median(sorted_data)

lower_half = sorted_data[sorted_data < median]
upper_half = sorted_data[sorted_data > median]

q1 = np.median(lower_half)
q3 = np.median(upper_half)

print(min, q1, median, q3, max)

27 82.0 112.5 134.0 155


IQR = Q3- Q1

In [38]:
iqr  = q3 - q1
iqr

np.float64(52.0)

In [40]:
# 식으로 구현
data = np.array([155,126,27,82,115,140,73,92,110,134])
sorted_data = np.sort(data)

n = len(data)

q3_percentile = (n-1) * 0.75 + 1
j3, h3 = divmod(q3_percentile, 1)
q3 = (1-h3) * sorted_data[int(j3) - 1] + h3 * sorted_data[int(j3)]
q3

np.float64(132.0)

In [41]:
import numpy as np

data = np.array([155,126,27,82,115,140,73,92,110,134])

q1 = np.percentile(data, 25)
q2= np.percentile(data, 50)
q3 = np.percentile(data, 75)

print(q1, q2, q3)


84.5 112.5 132.0


In [42]:
#데이터에 대응하는 백분위 구하기
import numpy as np
import scipy.stats as sp

percentiles = np.array([sp.percentileofscore(data, i ,kind = 'rank') for i in data])
print(percentiles)

[100.  70.  10.  30.  60.  90.  20.  40.  50.  80.]


샤피로윌크 검정 : shapiro(data)

- 표본 크기가 50개 이하인 작은 데이터들의 정규성 검정을 위해 고안된 방법
- 매우 민감해서 데이터의 분포가 타겟 분포와 조금만 달라도 p-val가 작게 나와 귀무가서링 기각될 수 있다

카이제곱 검정
- 표본 분산 검정
- 카이젭곱 독립성 검정
- 카이제곱 동질성 검정
- 카이제곱 적합도 검정

In [43]:
import numpy as np

sample = [10.67, 9.92, 9.62, 9.53, 9.14, 9.74, 8.45, 12.65, 11.47, 8.62]
n = len(sample)
var = np.var(sample, ddof = 1)
t = (n-1) * (var) / 1.3

In [44]:
t

np.float64(11.626530769230774)

In [47]:
from scipy.stats import chi2

p = 1 - chi2.cdf(t, df = n-1)
p

np.float64(0.23519165145589116)